## Introduction to `robotoc` 
# 1: Getting Started

## Installation (Ubuntu)
1. Please make sure that your OS is Ubuntu 22.04, 20.04, or 18.04
2. Install build tools via 
```
sudo apt install build-essential
```
3. Install `Eigen3` via
```
sudo apt install libeigen3-dev
```
4. Install [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio) and its viewer ([`gepeto-viewer-corba`](https://github.com/Gepetto/gepetto-viewer-corba)) (see [here](https://stack-of-tasks.github.io/pinocchio/download.html) for detailed instruction)
  - Ubuntu 22.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py310-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py310-qt5-gepetto-viewer-corba -y
```
  - Ubuntu 20.04:
```
sudo apt install -qqy lsb-release gnupg2 curl -y
echo "deb [arch=amd64] http://robotpkg.openrobots.org/packages/debian/pub $(lsb_release -cs) robotpkg" | sudo tee /etc/apt/sources.list.d/robotpkg.list
curl http://robotpkg.openrobots.org/packages/debian/robotpkg.key | sudo apt-key add -
sudo apt update -y
sudo apt install -qqy robotpkg-py38-pinocchio -y
echo export PATH=/opt/openrobots/bin:$PATH >> ~/.bashrc
echo export PKG_CONFIG_PATH=/opt/openrobots/lib/pkgconfig:$PKG_CONFIG_PATH >> ~/.bashrc
echo export LD_LIBRARY_PATH=/opt/openrobots/lib:$LD_LIBRARY_PATH >> ~/.bashrc
echo export PYTHONPATH=/opt/openrobots/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
echo export CMAKE_PREFIX_PATH=/opt/openrobots:$CMAKE_PREFIX_PATH >> ~/.bashrc
sudo apt update && sudo apt install robotpkg-py38-qt5-gepetto-viewer-corba -y
```
5. Install `robotoc` via
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON
sudo make install -j$(nproc)
```
  and set Python PATH 
  - Ubuntu 22.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.10/site-packages:$PYTHONPATH >> ~/.bashrc
```
  - Ubuntu 20.04:
```
echo export PYTHONPATH=/usr/local/lib/python3.8/site-packages:$PYTHONPATH >> ~/.bashrc
```
6. Install `pybullet` and `meshcat` via
```
pip3 install pybullet meshcat
```

## Installation (Mac)
The following instructions are not well-tested and probably do not work.   
Therefore, I'd like to recommend using Ubuntu.   
1. Install `Pinocchio` 
```
brew tap gepetto/homebrew-gepetto
brew install pinocchio
```
2. Check the full-path to `g++`. (this is necessary because the default compiler is clang in Mac)
```
ls -l /usr/local/bin | grep g++
```
3. Install `robotoc` via
```
git clone https://github.com/mayataka/robotoc
cd robotoc
mkdir build && cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DOPTIMIZE_FOR_NATIVE=ON -DCMAKE_CXX_COMPILER=FULL_PATH_TO_GPLUSPLUS
sudo make install -j$(nproc)
```
  and set Python PATH 
```
echo export PYTHONPATH=/usr/local/lib/python3.9/site-packages:$PYTHONPATH >> ~/.zshrc
```
4. Install `pybullet` and `meshcat` via
```
pip3 install pybullet meshcat
```

## Test Whole-body MPC
To verify the installation was successful, we run a whole-body MPC simulation.   
The other notebooks instruct the details of `robotoc` API.

### Import modules

In [ ]:
import robotoc
import numpy as np

### Create robot

In [ ]:
model_info = robotoc.RobotModelInfo()
model_info.urdf_path = 'urdf/a1_description/urdf/a1.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.05
model_info.point_contacts = [robotoc.ContactModelInfo('FL_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('RL_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('FR_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('RR_foot', baumgarte_time_step)]
robot = robotoc.Robot(model_info)

### Create whole-body MPC

In [ ]:
T = 0.5
N = 20
mpc = robotoc.MPCFlyingTrot(robot, T, N)

### Commands sent to whole-body MPC (`vcom_cmd`, `yaw_rate_cmd`)

In [ ]:
# cmd_type = 'forward'
# cmd_type = 'backward'
# cmd_type = 'side'
# cmd_type = 'curve'
cmd_type = 'rotation'

if cmd_type == 'forward':
    step_length = np.array([0.2, 0.0, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'backward':
    step_length = np.array([-0.1, 0.0, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'side':
    step_length = np.array([0.0, 0.15, 0.0]) 
    step_yaw = 0.
elif cmd_type == 'curve':
    step_length = np.array([0.1, 0.0, 0.0]) 
    step_yaw = np.pi / 30.
elif cmd_type == 'rotation':
    step_length = np.array([0.0, 0.0, 0.0]) 
    step_yaw = np.pi / 20.
    
step_height = 0.1
stance_time = 0.15
flying_time = 0.1
swing_start_time = 0.5

vcom_cmd = 0.5 * step_length / (flying_time+stance_time)
yaw_rate_cmd = step_yaw / flying_time

### Create a foot step planner

In [ ]:
planner = robotoc.FlyingTrotFootStepPlanner(robot)
raibert_gain = 0.9
planner.set_raibert_gait_pattern(vcom_cmd, yaw_rate_cmd, flying_time, stance_time, raibert_gain)
mpc.set_gait_pattern(planner, step_height, flying_time, stance_time, swing_start_time)

### Create initial state (`q`, `v`) and initial time (`t`)

In [ ]:
q = np.array([0, 0, 0.3181, 0, 0, 0, 1, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3, 
              0.0,  0.67, -1.3])
v = np.zeros(robot.dimv())
t = 0.0

### Initialize MPC (performs few Newton-type iterations)

In [ ]:
option_init = robotoc.SolverOptions()
option_init.max_iter = 10
option_init.nthreads = 4
mpc.init(t, q, v, option_init)

### Set solver options to MPC (number of the Newton-type iterations at each sampling time, number of threads used in parallel computations)

In [ ]:
option_mpc = robotoc.SolverOptions()
option_mpc.max_iter = 2 # MPC iterations
option_mpc.nthreads = 4
mpc.set_solver_options(option_mpc)

### Create a simulator and run the simulation!

In [ ]:
from a1_simulator import A1Simulator
from robotoc_sim import CameraSettings, MPCSimulation

time_step = 0.0025 # 400 Hz MPC
t0 = 0.0
q0 = q.copy()
simulation_time = 5.0

a1_simulator = A1Simulator(urdf_path=model_info.urdf_path, time_step=time_step)
camera_settings = CameraSettings(camera_distance=2.0, camera_yaw=45, camera_pitch=-10.0, 
                                 camera_target_pos=q0[0:3]+np.array([0.1, 0.5, 0.0]))
a1_simulator.set_camera_settings(camera_settings=camera_settings)

simulation = MPCSimulation(simulator=a1_simulator)
simulation.run(mpc=mpc, t0=t0, q0=q0, simulation_time=simulation_time, 
               feedback_delay=True, verbose=False, log=True, name='a1_flying_trot')
a1_simulator.disconnect()

### Plots the simulation results
(`matplotlib` and `seaborn` are required, e.g., via `pip install matplotlib seaborn`)

In [ ]:
q_log = np.genfromtxt(simulation.q_log)
v_log = np.genfromtxt(simulation.v_log)
t_log = np.genfromtxt(simulation.t_log)
sim_steps = t_log.shape[0]

vcom_log = []
wcom_log = []
vcom_cmd_log = []
yaw_rate_cmd_log = []
for i in range(sim_steps):
    R = robotoc.utils.rotation_matrix_from_quaternion(q_log[i][3:7])
    robot.forward_kinematics(q_log[i], v_log[i])
    vcom_log.append(R.T@robot.com_velocity()) # robot.com_velocity() is expressed in the world coordinate
    wcom_log.append(v_log[i][3:6])
    vcom_cmd_log.append(vcom_cmd)
    yaw_rate_cmd_log.append(yaw_rate_cmd)

%matplotlib inline
plot_mpc = robotoc.utils.PlotCoMVelocity()
plot_mpc.plot(t_log, vcom_log, wcom_log, vcom_cmd_log, yaw_rate_cmd_log, simulation.name)